# 슈카월드 컨텐츠를 기반으로 채팅하기







## Semantic Searching
  - 데이터를 로드합니다.
  - 질문의 Embedding 을 계산합니다.
  - 계산된 Embedding 과 유사한 결과들을 탐색합니다.
  - (참조) https://platform.openai.com/docs/guides/embeddings/what-are-embeddings

In [ ]:
import numpy as np
import pandas as pd
import openai
import tiktoken

In [ ]:
openai.api_key = OPENAI_API_KEY

In [ ]:
datafile_path = "./data/shuka_embeddings.csv"

df = pd.read_csv(datafile_path)
df["embedding"] = df.embedding.apply(eval).apply(np.array)
df

In [ ]:
from openai.embeddings_utils import get_embedding, cosine_similarity

# 연관된 영상을 검색합니다.
def search_video(df, product_description, n=3, pprint=True):
    product_embedding = get_embedding(
        product_description,
        engine="text-embedding-ada-002"
    )
    df["similarity"] = df.embedding.apply(lambda x: cosine_similarity(x, product_embedding))

    results = (
        df.sort_values("similarity", ascending=False)
        .head(n)
    )

    return results

In [ ]:
# 하고 싶은 말을 입력하고, 비슷한 결과들을 열람합니다.

results = search_video(df, "러시아와 우크라이나의 전쟁은 어떻게 되어가고 있나요?", n=5)
results

## 검색 결과와 함께 질문하기!
  - 질문과 유사한 컨텐츠를 검색합니다.
  - GPT 의 프롬프트에 유사한 결과를 함께 넣어 줍니다.
  - GPT 가 학습할 때 없었던 정보이지만, 이렇게 같이 넣어주면 GPT 가 누락되지 않은 정보를 이용하여 대답을 할 수 있습니다.

In [ ]:
GPT_MODEL="gpt-4"
def num_tokens(text: str, model: str = GPT_MODEL) -> int:
    """Return the number of tokens in a string."""
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))


def query_message(
    query: str,
    df: pd.DataFrame,
    model: str,
    token_budget: int
) -> str:
    """Return a message for GPT, with relevant source texts pulled from a dataframe."""
    
    results = search_video(df, query, n=5)
    strings = results['combined']
    
    introduction = 'Use the below articles on the 슈카월드 to answer the subsequent question. If the answer cannot be found in the articles, write "I could not find an answer."'
    question = f"\n\nQuestion: {query}"
    message = introduction
    for string in strings:
        next_article = f'\n\n:\n"""\n{string}\n"""'
        print("Tokens in Use : ", num_tokens(message + next_article + question, model=model))
        if (
            num_tokens(message + next_article + question, model=model)
            > token_budget
        ):
            
            break
        else:
            message += next_article
    return message + question

def ask(
    query: str,
    df: pd.DataFrame = df,
    model: str = GPT_MODEL,
    token_budget: int = 8192 - 500,
    print_message: bool = False,
) -> str:
    """Answers a query using GPT and a dataframe of relevant texts and embeddings."""
    message = query_message(query, df, model=model, token_budget=token_budget)
    if print_message:
        print(message)
    messages = [
        {"role": "system", "content": "You answer questions about 슈카의 유투브 내용"},
        {"role": "user", "content": message},
    ]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0
    )
    response_message = response["choices"][0]["message"]["content"]
    return response_message

In [ ]:
# set print_message=True to see the source text GPT was working off of
ask('우크라이나와 러시아의 전쟁은 어떻게 되어가고 있나요?', model="gpt-4", print_message=True)